<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds datetime
import pandas as pd, numpy as np, wrds, pytz
import datetime as dt

# Read awards from csv

In [2]:
wsb = pd.read_csv('/content/r_wallstreetbets_posts.csv',index_col='id',low_memory=False)

In [3]:
wsb = wsb[['total_awards_received']]

# Data cleaning
* Collect and summarize data by day and ticker
* Convert UTC to US/Eastern
* Merge award data from WSB

In [14]:
def comb_tic(x):
    if len(x[1]) == 0:
        return x[0]
    elif len(x[0]) == 0:
        return x[1]
    else:
        return list(set(x[0]).union(x[1]))

for yr in range(2012,2021,1):
    ticker = pd.read_parquet(f'/content/ticker_{yr}.parquet.gzip')
    col = ['date_utc','num_comments','score',
            'ticker_a','ticker_b']
    ticker = ticker[col]
    ticker['comb_tic'] = ticker[['ticker_a','ticker_b']].apply(comb_tic, axis=1)
    ticker['add_col'] = ticker['comb_tic'].apply(len)
    pos = len(ticker.columns)
    tics = []
    for i in range(pos, ticker['add_col'].max() + pos, 1):
        tics = tics + [f'tic{i - pos}']
    ticker[tics] = pd.DataFrame(ticker['comb_tic'].to_list(), index=ticker.index)
    ticker.drop(['ticker_a','ticker_b','comb_tic'],inplace=True, axis=1)
    ticker['datetime'] = ticker['date_utc'].dt.tz_convert('US/Eastern')
    start = dt.time(16,0,0)
    end = dt.time(23,59,59)
    ticker['date'] = np.where(ticker['datetime'].dt.time.between(start,end),
                            ticker['datetime'].dt.date + pd.DateOffset(days=1), 
                            ticker['datetime'].dt.date + pd.DateOffset(days=0))
    ticker.drop(['date_utc','datetime','add_col'],axis=1,inplace=True)
    ticker = ticker.merge(wsb,how='left',left_index=True,right_index=True)
    tic_t = pd.wide_to_long(ticker.reset_index(),'tic',i='id',j='num')[lambda x: ~x['tic'].isna()]
    tic_t.reset_index(inplace=True)
    tic_t.rename({'total_awards_received':'awards','id':'posts'},inplace=True,axis=1)
    tic_t = tic_t.reset_index().groupby(['date','tic']).agg({'score':'sum',
                                                    'num_comments':'sum',
                                                    'awards':'sum',
                                                    'posts':'count'
                                                    }).reset_index()
    tic_t.to_parquet(f'/content/tic_{yr}.parquet.gzip',compression='gzip')
    
    print(f'tic_{yr} saved.')    

tic_2012 saved.
tic_2013 saved.
tic_2014 saved.
tic_2015 saved.
tic_2016 saved.
tic_2017 saved.
tic_2018 saved.
tic_2019 saved.
tic_2020 saved.


In [27]:
ticker.shape

(300272, 108)

In [ ]:
ticker[ticker['']]

# Combine all data

In [15]:
dataset = list()
for i in range(2012,2021,1):
    dataset.append(f'tic_{i}')
tic = pd.DataFrame()
for yr in dataset:
    vars()[yr] = pd.read_parquet(f'{yr}.parquet.gzip')
    tic = tic.append(vars()[yr])

In [16]:
tic.shape

(222840, 6)

In [20]:
tic.to_csv('wsb_ticker.csv')

In [25]:
tic.describe()

,score,num_comments,awards,posts
count,222840.000000,222840.000000,222840.000000,222840.000000
mean,39.295903,57.736856,0.017887,1.888130
std,421.764252,552.615789,0.416889,5.027867
min,0.000000,0.000000,0.000000,1.000000
25%,1.000000,3.000000,0.000000,1.000000
50%,2.000000,14.000000,0.000000,1.000000
75%,12.000000,40.000000,0.000000,2.000000
max,50035.000000,87207.000000,87.000000,1089.000000


# Variable description
`score`,`num_comments`,`awards`, and `posts` are derived from individual posts summarized at the `date` and `tic` (ticker) level. Values of these variables repeating for multiple observations is likely an indication that the values came from the same single (set of) post (posts).

Number of observations: 222,840

|       |         score |  num_comments |        awards |         posts |
|------:|--------------:|--------------:|--------------:|--------------:|
| count | 222840.000000 | 222840.000000 | 222840.000000 | 222840.000000 |
|  mean |     39.295903 |     57.736856 |      0.017887 |      1.888130 |
|  std  |    421.764252 |    552.615789 |      0.416889 |      5.027867 |
|  min  |      0.000000 |      0.000000 |      0.000000 |      1.000000 |
|  25%  |      1.000000 |      3.000000 |      0.000000 |      1.000000 |
|  50%  |      2.000000 |     14.000000 |      0.000000 |      1.000000 |
|  75%  |     12.000000 |     40.000000 |      0.000000 |      2.000000 |
|  max  |  50035.000000 |  87207.000000 |     87.000000 |   1089.000000 |

##  `date`
The date portion extracted from `date_utc` tag attached to Reddit posts after converting the timezone to *'US/Eastern'*. If the time portion is between 4:00:00 PM and 11:59:59 PM EST/EDT, the date portion is incremented by one day to reflect option market hours (9:30 AM through 4:00 PM).

Min `date`: 2012-04-11
Max `date`: 2021-01-01

## `score`
Upvotes individual posts received summed at the `date` and `tic` level.

## `num_comments`
Comments individual posts received summed at the `date` and `tic` level.
 
## `awards`
Awards individual posts received summed at the `date` and `tic` level.

## `posts`
Number of posts that mention `tic` on `date`.
